### Members of FOCS's project
<ul>
    <li>Emanuele Marnati, matricola 812503 
    <li>Lilia Grasso, matricola 813210
        </ul>

# Project development 

#### We commented all the points both in English and Italian

# Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import re

## Datasets

In [2]:
school_country = pd.read_csv("school_and_country_table.csv")
cwur = pd.read_csv("cwurData.csv")
attainment = pd.read_csv("educational_attainment_supplementary_data.csv")
times = pd.read_csv("timesData.csv")
shanghai = pd.read_csv("shanghaiData.csv")

##### timer for the execution

In [3]:
start = time.time()  

# Points development

# Preprocessing

#### Considering the fact that in the following points we need to use the column "income" of the datasets that we are going to create below, after having found out that this column is a string we casted "to_numeric" to convert it into a numeric type. Furthermore then noticed some descrepancies in the country name of times dataset, we decided to solve that.

##### Considerando il fatto che nei punti seguenti abbiamo bisogno di usare la colonna "reddito" dei dataset che creeremo di seguito e dopo aver scoperto che questa colonna è di tipo stinga, abbiamo fatto il cast "to_numeric" per convertire la stinga in un tipo numerico. Abbiamo poi notato alcune discrepanze nel nome del paese del dataset di volte, abbiamo deciso di risolvere questo problema.

In [4]:
times.dtypes

world_rank                 object
university_name            object
country                    object
teaching                  float64
international              object
research                  float64
citations                 float64
income                     object
total_score                object
num_students               object
student_staff_ratio       float64
international_students     object
female_male_ratio          object
year                        int64
dtype: object

In [5]:
times['income'] = pd.to_numeric(times['income'], errors="coerce") 
times['income']

0       34.5
1       83.7
2       87.5
3       64.3
4        NaN
        ... 
2598    35.4
2599    44.0
2600    37.9
2601    40.4
2602    39.8
Name: income, Length: 2603, dtype: float64

In [6]:
times['country'] = times['country'].str.replace('Unisted', 'United')
times['country'] = times['country'].str.replace('Unted', 'United')

 ## 1)For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

#### The idea behind our code is that we are interested in the most and less recent time that a university has been ranked. For obtaining two dataset after groupping by "university_name" we used "idxmax" to find the university whose year of ranking is the maximum  and "idxmin" to find the universities whose year of ranking is the minimum, and to access a group of rows and columns by that labels we used ".loc". 

##### L'idea dietro il nostro codice è che siamo interessati al tempo più e meno recente in cui un'università è stata classificata. Per ottenere due set di dati dopo il raggruppamento per "university_name" abbiamo usato "idxmax" per trovare l'università il cui anno di classifica è il massimo e "idxmin" per trovare le università il cui anno di classifica è il minimo, e per accedere a un gruppo di righe e colonne da quelle etichette abbiamo usato ".loc". 

In [7]:
new = times.loc[times.groupby('university_name')['year'].idxmax()] 
old = times.loc[times.groupby('university_name')['year'].idxmin()]

In [8]:
new

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20%,46 : 54,2016
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,"2,218",8.0,14%,49 : 51,2016
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30%,18 : 82,2016
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54%,27 : 73,2016


In [9]:
old

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,"35,569",17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,"2,400",7.9,20%,46 : 54,2011
99,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,"2,218",8.0,14%,49 : 51,2011
38,39,École Polytechnique,France,57.9,77.9,56.1,91.4,NaN,69.5,"2,429",4.8,30%,18 : 82,2011
47,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,"9,666",10.5,54%,27 : 73,2011


## 2)For each university, compute the improvement in income between the least recent and the most recent data points


#### We merged the two dataset created in point 1, to obtain a dataset where there are, for each university:
<lu>
<li>A column with the less recent year of ranking (year_x) and a column with the income associated at that year (income_x)
<li>A column with the most recent year (year_y) of ranking and a column with the income associated to that year (income_y)
    

##### Abbiamo unito i due dataset creati al punto 1, per ottenere un dataset dove ci sono, per ogni università:
<lu>
<li>Una colonna con l'anno meno recente della classifica (year_x) e una colonna con il reddito associato a quell'anno(income_x)
<li>Una colonna con l'anno più recente (year_y) della classifica e una colonna con il reddito associato a quell'anno (income_y)

In [10]:
improvement = pd.merge(old,new,on='university_name') 
improvement

,world_rank_x,university_name,country_x,teaching_x,international_x,research_x,citations_x,income_x,total_score_x,num_students_x,...,international_y,research_y,citations_y,income_y,total_score_y,num_students_y,student_staff_ratio_y,international_students_y,female_male_ratio_y,year_y
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,"35,569",...,17.9,3.7,35.7,NaN,-,"35,569",17.0,1%,-,2016
1,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",...,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",...,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
3,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",...,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
4,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",...,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,"2,400",...,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20%,46 : 54,2016
814,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,"2,218",...,65.6,30.0,69.0,31.7,-,"2,218",8.0,14%,49 : 51,2016
815,39,École Polytechnique,France,57.9,77.9,56.1,91.4,NaN,69.5,"2,429",...,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30%,18 : 82,2016
816,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,"9,666",...,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54%,27 : 73,2016


#### We have created a new column whose value per each row is the difference between the income of the most recent year and the income of the less recent year. Some income are negative because there has been a worsening over time.

##### Abbiamo creato una nuova colonna il cui valore per ogni riga è la differenza tra il reddito dell'anno più recente e il reddito dell'anno meno recente. Alcuni redditi sono negativi perché c'è stato un peggioramento nel tempo.

In [11]:
improvement['improvement'] = improvement['income_y']-improvement['income_x']
improvement['improvement']

0       NaN
1       7.3
2      -0.3
3       6.8
4      -4.2
       ... 
813     6.4
814     5.6
815     NaN
816    27.4
817     0.0
Name: improvement, Length: 818, dtype: float64

## 3)Find the university with the largest increase computed in the previous point


#### To find the university with the largest increase we have queried the dataset looking for the row whose improvement was the maximum. This university, according to this ranking is "Technische Universität Dresden"

##### Per trovare l'università con il maggior aumento abbiamo interrogato il dataset cercando la riga il cui miglioramento era il massimo. Questa università, secondo questa classifica è "Technische Universität Dresden"

In [12]:
improvement[improvement['improvement']>=improvement['improvement'].max()]

,world_rank_x,university_name,country_x,teaching_x,international_x,research_x,citations_x,income_x,total_score_x,num_students_x,...,research_y,citations_y,income_y,total_score_y,num_students_y,student_staff_ratio_y,international_students_y,female_male_ratio_y,year_y,improvement
428,251-275,TU Dresden,Germany,27.3,49.2,13.8,57.4,31.9,-,"35,487",...,45.8,66.1,99.7,52.1,"35,487",37.4,12%,42 : 58,2016,67.8


#### The ".max()" in the case that the maximum values are more than one and all equals between them, it doesn't give all of them, so we groupped for university name and used "sort_values(ascending=False)" to have the values sorted by the maximum to the minimum and to check our query

##### Il ".max()" nel caso in cui i valori massimi siano più di uno e tutti uguali tra loro, non li dà tutti, quindi abbiamo fatto un raggruppamento per nome dell'università e usato "sort_values(ascending=False)" per avere i valori ordinati dal massimo al minimo e per controllare la nostra query

In [13]:
improvement.groupby("university_name")['improvement'].max().sort_values(ascending= False)

university_name
TU Dresden                           67.8
Nanyang Technological University     59.9
LMU Munich                           59.6
Karlsruhe Institute of Technology    59.5
Nagoya University                    58.3
                                     ... 
William & Mary                        NaN
Xiamen University                     NaN
Yale University                       NaN
Yeshiva University                    NaN
École Polytechnique                   NaN
Name: improvement, Length: 818, dtype: float64

## 4)For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range


#### Some university in shanghai dataset and times dataset have an acronim near the name (for example "(MIT)" in the shanghai dataset), so we have built a function who looks for the brackets near the name, looking for the regular expression compiled in "pare" and, once found, keeps only the university's name. We then cleaned with pandas functions the universities names we've found out.  

##### Alcune università nel dataset shanghai e nel dataset times hanno un acronimo vicino al nome (per esempio "(MIT)" nel dataset shanghai), così abbiamo costruito una funzione che cerca le parentesi vicino al nome, cercando l'espressione regolare compilata in "coppia" e, una volta trovata, mantiene solo il nome dell'università. Abbiamo poi pulito con funzioni pandas i nomi delle università che abbiamo scoperto.

In [14]:
print(f" {times.loc[2]['university_name']} is different from  {shanghai.loc[4]['university_name']}")

 Massachusetts Institute of Technology is different from  Massachusetts Institute of Technology (MIT)


In [15]:
pare = re.compile('^(.+)(\s\(\w+[- ]?\w+?\))$')
def sost_pare(elem):
    t = pare.search(elem)
    if t:
        t1 = t.group(2)
        t3 = t.group(1)
        return t3
    else:
        return elem
#apply the function
cwur['institution_corr'] = cwur['institution'].astype(str).apply(sost_pare)    
shanghai['university_name_corr'] = shanghai['university_name'].astype(str).apply(sost_pare)
times['university_name_corr'] = times['university_name'].astype(str).apply(sost_pare)
new['university_name_corr'] =new['university_name'].astype(str).apply(sost_pare)

#### We modified the strings as follow:
<lu>
<li> we've replaced the punctuation with one space
<li> we've replaced the underscore character with one space
<li> we've replaced consequent spaces with one space
<li> we've finally lowered the whole string

##### Abbiamo modificato le stringhe come segue:
<lu>
<li> abbiamo sostituito la punteggiatura con uno spazio
<li> abbiamo sostituito il carattere underscore con uno spazio
<li> abbiamo sostituito spazi in fila con uno spazio
<li> abbiamo in fine reso minuscola l'intera stringa

In [16]:
print(f" {times.loc[836]['university_name']} is different from  {shanghai.loc[4326]['university_name']}")

 Mines ParisTech is different from  MINES ParisTech


In [17]:
#cleaning the strings after further observations
times['university_name_corr'] = times['university_name_corr'].str.replace('[^\w\s]',' ').str.replace('_',' ').str.replace(' +',' ').str.strip().str.lower()
shanghai['university_name_corr'] = shanghai['university_name_corr'].str.replace('[^\w\s]',' ').str.replace('_',' ').str.replace(' +',' ').str.strip().str.lower()
cwur['institution_corr'] = cwur['institution_corr'].str.replace('[^\w\s]',' ').str.replace('_',' ').str.replace(' +',' ').str.strip().str.lower()
new['university_name_corr'] = new['university_name_corr'].str.replace('[^\w\s]',' ').str.replace('_',' ').str.replace(' +',' ').str.strip().str.lower()

#### Some ranking in the shanghai and times dataframes are expressed as a range and some have an "=" before the rank. We've written a regular expression that looks for these two types of strings ("interval" for intervals and "equals" for the "=" symbol before the ranking).Then we've defined a function that if "interval" occurs it calculates the mean, else if "equals" occurs replace the symbol with a space, otherwise when no regular expression occurs it returns the rank as it is.

##### Alcune classifiche nei dataframe shanghai e times sono espresse come intervallo e alcune hanno un "=" prima della classifica. Abbiamo scritto un'espressione regolare che cerca questi due tipi di stringhe ("interval" per gli intervalli e "equals" per il simbolo "=" prima della classifica), poi abbiamo definito una funzione che se si verifica "interval" calcola la media, altrimenti se si verifica "equals" sostituisce il simbolo con uno spazio, altrimenti quando non si verifica alcuna espressione regolare restituisce la classifica come è.


In [18]:
interval = re.compile('^(\d+)\-(\d+)$')
equals = re.compile('^(\=)(\d+)')

def lettura(elem):
    m=interval.search(elem)
    l=equals.search(elem)
    if m:
        smallr = int(m.group(1))
        bigr = int(m.group(2))
        return((bigr+smallr)/2)
    elif l:
        uguale = l.group(1)
        numero = l.group(2)
        lpulito= uguale.replace('=','')
        return int(lpulito+numero) 
    else:
        return float(elem)
    
new['ranking']=new['world_rank'].apply(lettura) 
shanghai['ranking'] = shanghai['world_rank'].apply(lettura)
times['ranking']=times['world_rank'].apply(lettura)

#### To obtain the most recent data for shanghai and cwur we used the same code of point 1

##### Per ottenere i dati più recenti per shanghai e cwur abbiamo usato lo stesso codice del punto 1

In [19]:
newshanghai = shanghai.loc[shanghai.groupby('university_name_corr')['year'].idxmax()] 
newcwur = cwur.loc[cwur.groupby('institution_corr')['year'].idxmax()] 

 #### We merged all three dataframes of ranking so that per each university we have a column with the value of the cwur's ranking, one for the times' ranking and one for shanghai's ranking.

##### Abbiamo unito tutti e tre i dataframes di ranking in modo che per ogni università abbiamo una colonna con il valore del ranking di cwur, una per il ranking di times e una per il ranking di shanghai.

In [20]:
allrankings = pd.merge(new, newcwur, left_on='university_name_corr', right_on='institution_corr')
allrankings = pd.merge(allrankings, newshanghai, on='university_name_corr')

#rename the columns to have a light dataset
allrankings = allrankings.rename(columns={'ranking_x':'new_times','world_rank_y':'cwur','ranking_y':'shangai'})

#keeps only the relevant columns
allrankings = allrankings[['university_name_corr','year','new_times','cwur','shangai']]
allrankings

,university_name_corr,year,new_times,cwur,shangai
0,aalborg university,2015,225.5,565,350.5
1,aalto university,2015,275.5,421,450.5
2,aarhus university,2015,106.0,122,73.0
3,aix marseille university,2015,275.5,206,125.5
4,aristotle university of thessaloniki,2015,700.5,459,450.5
...,...,...,...,...,...
384,yale university,2015,12.0,11,11.0
385,yeshiva university,2015,164.0,171,250.5
386,yonsei university,2015,325.5,98,250.5
387,york university,2014,325.5,337,450.5


#### To calculate the maximum difference between the rankings we defined a function that we apply only to the column of interest (the ranking's one). This function puts in into a list the differences between the value of the rankings of a given row and then it returns the absolute maximum value between them. We get 'aarhus university'

##### Per calcolare la differenza massima tra le classifiche abbiamo definito una funzione che applichiamo solo alla colonna di interesse (quella della classifica). Questa funzione mette in una lista le differenze tra i valori delle classifiche di una data riga e poi restituisce il valore massimo assoluto tra loro. Otteniamo 'aarhus university'.

In [21]:
def colonna(col1, col2,col3):
    l = [col2-col1, col3-col2, col3 - col1]
    return abs(max(l))

allrankings['max_difference']= allrankings.apply(lambda x: colonna(x.new_times,x.shangai,x.cwur), axis=1)
allrankings.head()

,university_name_corr,year,new_times,cwur,shangai,max_difference
0,aalborg university,2015,225.5,565,350.5,339.5
1,aalto university,2015,275.5,421,450.5,175.0
2,aarhus university,2015,106.0,122,73.0,49.0
3,aix marseille university,2015,275.5,206,125.5,80.5
4,aristotle university of thessaloniki,2015,700.5,459,450.5,8.5


In [22]:
allrankings[allrankings['university_name_corr']=='aarhus university']

,university_name_corr,year,new_times,cwur,shangai,max_difference
2,aarhus university,2015,106.0,122,73.0,49.0


## 5)Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

#### Given that in the column "num_students" the numbers of student are expressed as a string with "," as a separator, we have writte a function. This function after "students" occurs (it is a regular expression who looks for a string that is "numeric charachters,numeric charachters" splitted into two groups divided by "," where the first represents the thousands and the second represents the hundrends), it returns the int value obtained by the sum of thousands and hundrends.

##### Dato che nella colonna "num_students" i numeri degli studenti sono espressi come una stringa con "," come separatore, abbiamo scritto una funzione. Questa funzione dopo che compare "students" (è un'espressione regolare che cerca una stringa che sia "numeric charachters,numeric charachters" divisa in due gruppi da "," dove il primo rappresenta le migliaia e il secondo le centinaia), restituisce il valore int ottenuto dalla somma di migliaia e centinaia.


In [23]:
times['num_students']

0       20,152
1        2,243
2       11,074
3       15,596
4        7,929
         ...  
2598    21,958
2599    31,268
2600     4,122
2601    10,117
2602     8,663
Name: num_students, Length: 2603, dtype: object

In [24]:
studenti = re.compile('^(\d+)\,(\d+)$')

def stud(elem):
    j = studenti.search(elem)
    if j:
        migl = int(j.group(1))*(10**3)
        cent = int(j.group(2))
        return (migl+cent)
    else:
        return None

#apply the function to the columns
times['num_students_asint'] = times['num_students'].astype(str).apply(stud)
new['num_students_asint'] = new['num_students'].astype(str).apply(stud)
#universitybelowmean['num_students_asint'] = universitybelowmean['num_students'].astype(str).apply(stud)

#### We've written a function "femalemale" that after "ratio" regular expression is found, returns a list where the first element is the number of female and the second is the number of male. Furthermore we applied "femalemale" to the column "female_male_ratio" creating a pandas series from the results of the function so that the series with the first elements goes in the new column "female" and the series with all the second elements goes in the new column "male".

##### Abbiamo scritto una funzione "femalemale" che dopo aver trovato l'espressione regolare "ratio", restituisce una lista dove il primo elemento è il numero di femmine e il secondo è il numero di maschi. Inoltre abbiamo applicato "femalemale" alla colonna "female_male_ratio" creando una serie pandas dai risultati della funzione in modo che la serie con i primi elementi vada nella nuova colonna "female" e la serie con tutti i secondi elementi vada nella nuova colonna "male".

In [25]:
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [26]:
times.female_male_ratio

0           NaN
1       33 : 67
2       37 : 63
3       42 : 58
4       45 : 55
         ...   
2598    48 : 52
2599    36 : 64
2600        NaN
2601    28 : 72
2602    43 : 57
Name: female_male_ratio, Length: 2603, dtype: object

In [27]:
ratio = re.compile('^(\d+)\s\:\s(\d+)$')

def femalemale(elem,elem2):
    k = ratio.search(elem)
    if k:
        female = int(k.group(1))*(10**(-2))
        male = int(k.group(2))*(10**(-2))
        return [(female*elem2),(male*elem2)]
    else:
        return None
    
new[['female', 'male']] = new.apply(lambda row: pd.Series(femalemale(row.astype(str).female_male_ratio,row.num_students_asint)), axis=1)
times[['female', 'male']] = times.apply(lambda row: pd.Series(femalemale(row.astype(str).female_male_ratio,row.num_students_asint)), axis=1)


#### Groupping by country, we calculate the total number of female and male attending the universities in that particular country.

##### Per ogni paese, calcoliamo il numero totale di donne e uomini che frequentano le università di quel particolare paese.

In [28]:
new['female'] = new['female'].round() 

In [29]:
new['male'] = new['male'].round() 

In [30]:
new.groupby('country')['female'].sum()

country
Argentina                     67191.0
Australia                    391736.0
Austria                       68364.0
Bangladesh                    21323.0
Belarus                       20219.0
                              ...    
Uganda                        18670.0
Ukraine                       17846.0
United Arab Emirates           9516.0
United Kingdom               711815.0
United States of America    1601453.0
Name: female, Length: 70, dtype: float64

In [31]:
new.groupby('country')['male'].sum()

country
Argentina                     41182.0
Australia                    321640.0
Austria                       66113.0
Bangladesh                    41393.0
Belarus                        9084.0
                              ...    
Uganda                        18670.0
Ukraine                       19250.0
United Arab Emirates           4931.0
United Kingdom               613028.0
United States of America    1556324.0
Name: male, Length: 70, dtype: float64

## 6)Find the universities where the ratio between female and male is below the average ratio (computed over all universities)


#### We've written a function that checks if there are "NaN" values and it checks, when the value of men is a number, if it is "0". If so, it returns NaN value, otherwise it returns the ratio between female and male.

##### Abbiamo scritto una funzione che controlla se ci sono valori "NaN" e controlla, quando il valore degli uomini è un numero, se è 0. Se è così restituisce un valore "NaN", altrimenti restituisce il rapporto tra donne e uomini.

In [32]:
#We compile the regular expression that we want to find
number = re.compile('(^\d+\.\d)')

#function
def femalemaleratio(elem,elem2):
    male=number.search(elem)
    female=number.search(elem2)
    
    if (male and female):
        male1=float(male.group(1))
        female1=float(female.group(1))
        if (male1 != 0):
            return (female1/male1)
        else:
                return float('NaN')               

#we apply the function only at the columns 'male' and 'female' of dataset times           
times['ratiofemalemale'] = times.apply(lambda x: femalemaleratio(x.astype(str).male,x.astype(str).female), axis=1)
new['ratiofemalemale'] = new.apply(lambda x: femalemaleratio(x.astype(str).male,x.astype(str).female), axis=1)

#### We calculate the mean. Finally, we create a new dataset with the universities whose ratio was below the mean. 

##### Calcoliamo la media. Infine, creiamo un nuovo dataset con le università il cui rapporto era inferiore alla media. 

In [33]:
#calculate mean
mean = new['ratiofemalemale'].mean()

In [34]:
mean

1.0827813655601444

In [35]:
#create a dataframe where ratio is below
universitybelowmean = new[new['ratiofemalemale'] < mean]
universitybelowmean[['university_name_corr','female','male','ratiofemalemale']]

,university_name_corr,female,male,ratiofemalemale
2003,aalborg university,8363.0,9059.0,0.923170
2056,aalto university,5152.0,10947.0,0.470631
2105,aberystwyth university,4441.0,4811.0,0.923093
2406,ajou university,4193.0,8513.0,0.492541
2408,alexandria university,58618.0,68813.0,0.851845
...,...,...,...,...
2104,zhejiang university,19478.0,28030.0,0.694898
1856,école normale supérieure,1104.0,1296.0,0.851852
2013,école normale supérieure de lyon,1087.0,1131.0,0.961096
1904,école polytechnique,437.0,1992.0,0.219378


## 7)For each country compute the fraction of students that are in one of the universities computed in the previous point  (that is, the denominator of the ratio is the total number of students over all universities in the country). 


#### We firstly group the most recent universities by "country" in order to sum up the students for each university. We then join the previous resulting dataset (the universities below mean) with the "student_country" dataset. We then compute the ratio of student that attend uni below mean and the total number of student in that country.

##### Per prima cosa raggruppiamo le università più recenti per "paese" al fine di sommare gli studenti per ogni università. Poi uniamo il precedente datset risultante (le università sotto la media) con il dataset "student_country". Poi calcoliamo il rapporto tra gli studenti che frequentano le università sotto la media e il numero totale di studenti in quel paese.

In [36]:
student_country = new.groupby('country',as_index=False)['num_students_asint'].sum()

In [37]:
#calculate ratios
merging= pd.merge(universitybelowmean, student_country, on='country')
merging['ratios']=merging['num_students_asint_x']/merging['num_students_asint_y']
#show the relevant column
merging[['ratios','university_name_corr','country']]

,ratios,university_name_corr,country
0,0.145968,aalborg university,Denmark
1,0.153265,copenhagen business school,Denmark
2,0.083700,technical university of denmark,Denmark
3,0.151641,aalto university,Finland
4,0.032836,lappeenranta university of technology,Finland
...,...,...,...
339,1.000000,university of ibadan,Nigeria
340,1.000000,university of indonesia,Indonesia
341,1.000000,university of luxembourg,Luxembourg
342,1.000000,university of marrakech cadi ayyad,Morocco


## 8)Read the file `educational_attainment_supplementary_data.csv`, discarding any row with missing `country_name` or `series_name`

#### We used a "dropna" function selecting just the two columns of interest. 

##### Abbiamo usato una funzione "dropna" selezionando solo le due colonne di interesse. 

In [38]:
edu_without = attainment.dropna(subset=['country_name', 'series_name'])
edu_without

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


## 9)From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value


#### We used a melt function that manipulate the table structure from wide to long format, leaving out "id_vars" objects, creating "year" and associating "value" to each one, for each "id_vars" couple.
#### There are many null values. In order to not compromise the integrity of the data, it is decided to not intervene.

##### Abbiamo usato una funzione "melt" che manipola la struttura della tabella da formato largo a lungo, lasciando fuori gli oggetti "id_vars", creando "anno" e associando "valore" a ciascuno di essi, per ogni coppia "id_vars".
###### Ci sono molti valori nulli. Per non compromettere l'integrità dei dati, si decide di non intervenire.

In [39]:
edu_fancy = pd.melt(edu_without, id_vars=['country_name', 'series_name'], var_name = 'year', value_name = 'value')
edu_fancy

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


## 10)For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).


#### We created a new dataset with all of the universities using left join, in order to have all the institutions representing the ranking study.  We replaced the 'NaN' value with the string 'No ranking', in order to have a cleared dataset. We then counted the occurences of the uni whose name was different from 'no ranking'.

##### Abbiamo creato un nuovo dataset con tutte le università usando left join, in modo da avere tutte le istituzioni che rappresentano lo studio del ranking.  Abbiamo sostituito il valore 'NaN' con la stringa 'No ranking', in modo da avere un dataset pulito. Abbiamo poi contato le occorrenze dell'università il cui nome era diverso da 'no ranking'.

In [40]:
#left join to obtain all the universities 
mer = pd.merge(times, cwur,how='left', left_on='university_name_corr', right_on='institution_corr',sort=True)
mer = pd.merge(mer, shanghai, how='left', on='university_name_corr')

#rename the columns
mer = mer.rename(columns={'ranking_x':'times','world_rank_y':'cwur','world_rank':'shangai'})

# keep relevant cols
mer = mer[['university_name_corr','year','times','cwur','shangai']]

#from np.nan to 'NaN' string
mer['shangai'] = mer['shangai'].replace(np.nan,'No ranking')
mer['times'] = mer['times'].replace(np.nan,'No ranking')
mer['cwur'] = mer['cwur'].replace(np.nan,'No ranking')

In [41]:
#function to count the number of times a university appear
def counter(col1, col2,col3):    
    count = 0
    if (col1 != 'No ranking'):
        count += 1
    if (col2 !=  'No ranking'):
        count += 1
    if (col3 != 'No ranking'):
        count += 1
        
    return count

mer['counter']= mer.apply(lambda x: counter(x.times,x.cwur,x.shangai), axis=1)
mer

,university_name_corr,year,times,cwur,shangai,counter
0,aalborg university,2014.0,325.5,593,401-500,3
1,aalborg university,2015.0,325.5,593,301-400,3
2,aalborg university,2014.0,325.5,565,401-500,3
3,aalborg university,2015.0,325.5,565,301-400,3
4,aalborg university,2014.0,375.5,593,401-500,3
...,...,...,...,...,...,...
37928,école polytechnique fédérale de lausanne,NaN,37.0,No ranking,No ranking,1
37929,école polytechnique fédérale de lausanne,NaN,34.0,No ranking,No ranking,1
37930,école polytechnique fédérale de lausanne,NaN,31.0,No ranking,No ranking,1
37931,örebro university,NaN,325.5,656,No ranking,2


## 11)In the times ranking, compute the number of times each university appears

#### We make a copy of the column "university_name" in a new column "number_of_times" to use it in the groupby to whom we apply a "count()"  function to find out the number of times that a university appears in times dataset

##### Facciamo una copia della colonna "nome_università" in una nuova colonna "numero_di_volte" per usarla nel groupby a cui applichiamo una funzione "count()" per trovare il numero di volte che un'università appare nel dataset times

In [42]:
#copy of university name's columns
times['number_of_times'] = times['university_name_corr'].copy()

In [43]:
sizetimes = times.groupby('university_name_corr', as_index=False)['number_of_times'].count()
sizetimes

,university_name_corr,number_of_times
0,aalborg university,5
1,aalto university,5
2,aarhus university,6
3,aberystwyth university,5
4,adam mickiewicz university,1
...,...,...
813,école normale supérieure,6
814,école normale supérieure de lyon,6
815,école polytechnique,6
816,école polytechnique fédérale de lausanne,6


## 12)Find the universities that appear at most twice in the times ranking.


#### Print the dataset where 'number of times' is less or equal than 2

##### Stampa il dataset in cui il numero di volte è inferiore o uguale a 2

In [44]:
sizetimes[sizetimes['number_of_times'] <= 2]

,university_name_corr,number_of_times
4,adam mickiewicz university,1
5,agh university of science and technology,1
6,aix marseille university,1
7,ajou university,1
9,alexandru ioan cuza university,1
...,...,...
806,yokohama city university,1
807,yokohama national university,1
810,yuan ze university,2
811,yıldız technical university,1


## 13)The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

#### We merge the data frames and select the columns of intrest with respect the year.
#### Just Stanford University keep the position among rankings and years!

##### Uniamo i dataframe e selezioniamo le colonne di interesse rispetto all'anno.
##### Solo l'Università di Stanford mantiene la posizione tra le classifiche e gli anni!

In [45]:
allu = pd.merge(times, cwur, left_on=['university_name_corr', 'year'], right_on=['institution_corr', 'year'])
allu = pd.merge(allu, shanghai, on=['university_name_corr', 'year'])
#rename the columns
allu = allu.rename(columns={'ranking_x':'times','world_rank_y':'cwur','ranking_y':'shangai'})

# keep relevant cols
allu = allu[['university_name_corr','year','times','cwur','shangai']]

#looks for the university that respect the conditions
allu.loc[(allu['times'] == allu['shangai']) & (allu['cwur'] == allu['shangai']) & (allu['cwur'] == allu['times'])]

,university_name_corr,year,times,cwur,shangai
65,stanford university,2013,2.0,2,2.0


In [46]:
end = time.time()

print("You spent {} seconds on our code,".format(int((end-start))))
print("namely {} minutes".format(int((end-start)/60)))

You spent 12 seconds on our code,
namely 0 minutes
